In [ ]:
import os
import glob
import pandas as pd
from google.colab import drive

# 0) 드라이브 마운트
drive.mount("/content/drive")

# 1) 너 폴더 경로 (여기만 맞춰)
base_dir = "/content/drive/MyDrive/30만 데이터"

# 2) 출력 폴더 (원본 폴더 안에 생성)
output_dir = os.path.join(base_dir, "_extracted_5cols")
os.makedirs(output_dir, exist_ok=True)

# 3) 추출할 컬럼 5개 (고정)
required_cols = [
    "발급회원번호",
    "이용금액_신용_B0M",
    "잔액_B0M",
    "이용건수_신용_B0M",
    "기준년월"
]

# 4) 폴더 안 csv 전부 찾기
csv_files = sorted(glob.glob(os.path.join(base_dir, "*.csv")))
print("찾은 CSV 개수:", len(csv_files))
print("저장 폴더:", output_dir)

# 5) 인코딩 자동 감지(헤더만 읽어서 맞는 인코딩 찾기)
def detect_encoding(csv_path):
    enc_list = ["utf-8-sig", "utf-8", "cp949", "euc-kr"]
    for enc in enc_list:
        try:
            pd.read_csv(csv_path, nrows=0, encoding=enc)
            return enc
        except:
            pass
    return None

# 6) 파일마다: df_target_base 생성 -> CSV 저장 (절대 중단 X, NaN 생성 X)
ok = 0
skip = 0

for csv_path in csv_files:
    try:
        file_name = os.path.basename(csv_path)

        enc = detect_encoding(csv_path)
        if enc is None:
            print("❌ 인코딩 감지 실패(스킵):", file_name)
            skip += 1
            continue

        # 컬럼 존재 확인(헤더만)
        header_df = pd.read_csv(csv_path, nrows=0, encoding=enc)
        cols = header_df.columns.tolist()

        missing = []
        for c in required_cols:
            if c not in cols:
                missing.append(c)

        if len(missing) > 0:
            print("⚠ 컬럼 부족으로 스킵:", file_name, "| missing:", missing)
            skip += 1
            continue

        # ✅ 여기서 "추출된 컬럼만 포함된 새로운 데이터프레임" 생성
        df_target_base = pd.read_csv(csv_path, usecols=required_cols, encoding=enc, low_memory=False)

        # (선택) 정렬 원하면 주석 해제
        # df_target_base = df_target_base.sort_values(by=["발급회원번호", "기준년월"]).reset_index(drop=True)

        out_name = file_name.replace(".csv", "_5cols.csv")
        out_path = os.path.join(output_dir, out_name)

        df_target_base.to_csv(out_path, index=False, encoding="utf-8-sig")

        print("✅ 저장 완료:", out_name, "| shape:", df_target_base.shape)
        ok += 1

    except Exception as e:
        print("❌ 처리 실패(스킵하고 계속):", os.path.basename(csv_path), "|", str(e))
        skip += 1
        continue

print("=== 요약 ===")
print("성공:", ok)
print("스킵:", skip)
print("결과 폴더:", output_dir)
